In [76]:
import math

import torch
from torch import tensor
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf

from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error
import sklearn.datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
from plot import plot_prediction
import numpy as np

%matplotlib inline

from plot import *

In [77]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

dtype = torch.double
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Optimizing Test Functions with EIpS

In [78]:
torch.manual_seed(0)

# BO design points
x_ = torch.arange(-1., 1.0, 0.01)
y_ = torch.arange(-1., 1.0, 0.01)
x_axis, y_axis = torch.meshgrid(x_, y_)
grid = torch.rot90(torch.stack((x_axis, y_axis), 2))

X_ = grid.reshape(len(x_)*len(y_), 2)

## Ridge Regression

In [79]:
iris = ds.load_iris()
X = iris.data
y = iris.target

X_2d = X[:, 1:3]
X_2d = X_2d[y > 0]
y_2d = y[y > 0]
y_2d -= 1
X = X_2d
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = y_2d
# test train split
#X, y = ds.make_moons(200, noise=0.3, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# hyperparameter search space
gamma_min = -9.
gamma_max = 3 #3
llambda_min = -2
llambda_max = 10 #10
resolution = 13
gamma_space = (np.logspace(gamma_min, gamma_max, resolution, base=2))
llambda_space = (np.logspace(llambda_min, llambda_max, resolution, base=2))

def log_scale(x, min, max): # map from [-1, 1] to log2([min, max])
    return torch.exp2((max - min) * ((x+1)*0.5) + min)

def plot_KRR_space_test(scores, X_samples, gamma_space, llambda_space):
    gamma_scaled = log_scale(X_samples[:, 0], gamma_min, gamma_max)
    llambda_scaled = log_scale(X_samples[:, 1], llambda_min, llambda_max)

    p, ax = plt.subplots(figsize=(8, 6))
    ax.title.set_text("Validation accuracy")

    im=ax.imshow(
        np.rot90(scores),
        interpolation="nearest",
        cmap=plt.cm.hot,
        extent=[0, len(gamma_space), 0, len(llambda_space)]
    )
    plt.colorbar(im)

    ax.set_xticks(np.arange(len(gamma_space)))
    ax.set_xticklabels(gamma_space, rotation=45)
    ax.set_yticks(np.arange(len(llambda_space)))
    ax.set_yticklabels(llambda_space)

    plt.show()
    plt.figure(figsize=(8, 8))
    plt.scatter(gamma_scaled, llambda_scaled, c='green')
    plt.xlabel("gamma")
    plt.ylabel("lambda")
    plt.xscale('log', base=2)
    plt.xscale('log', base=2)
    plt.title("Sample Locations")
    plt.show()

def sample_regression(gamma, llambda, K=None, scale=True):
    if scale:
        gamma = log_scale(gamma, gamma_min, gamma_max)
        llambda = log_scale(llambda, llambda_min, llambda_max)

    gamma = float(gamma)
    llambda = float(llambda)
    # train
    kr = KernelRidge(alpha=llambda, gamma=gamma, kernel='precomputed')
    kernel = RBF(length_scale=gamma)
    if K is None:
        K = kernel.__call__(X_train)
    kr.fit(K, y_train)

    # test
    K_xX = kernel.__call__(X_test, Y=X_train)
    predictions = kr.predict(K_xX)
    score = mean_squared_error(y_test, predictions)

    return tensor(score), K

In [80]:
scores = torch.zeros((len(gamma_space), len(llambda_space)))
best_score = math.inf
best_candidate = [0, 0]
for ix, x_ax in enumerate(gamma_space):  # gamma
    for iy, y_ax in enumerate(llambda_space):  # llambda
        scores[ix][iy] = sample_regression(torch.tensor(x_ax),
                                           torch.tensor(y_ax),
                                           scale=False)[0]

        if scores[ix][iy] < best_score:
            best_score = scores[ix][iy]
            best_candidate = [x_ax, y_ax]

## Bayesian Optimization

In [ ]:
def sample_bo(n_iters:int, plot=False):
    bounds = tensor([[x_[0], y_[0]], [x_[-1], y_[-1]]])

    X_samples = tensor([[torch.rand((1)), torch.rand((1))]])
    y_samples = sample_regression(X_samples[:,0], X_samples[:,1])[0].reshape(-1, 1)
    print("=======================")
    print("initialization")
    print("candidate: ", X_samples[0])
    print("model score for candidate: ", float(y_samples[0]))

    for i in range(n_iters):
        print("=======================")
        print("iteration:      ", i)
        #print(X_samples)
        gp = SingleTaskGP(X_samples, y_samples)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_model(mll)

        EI = acqf.ExpectedImprovement(
            gp,
            y_samples.min(),
            maximize=False
        )

        # plot
        if plot:
            plot_bo(gp, EI, X_samples)

        best_candidate, acq_value = optimize_acqf(
            acq_function=EI,
            bounds=bounds,
            q=1,
            num_restarts=1,
            raw_samples=20
        )

        y_new_sample, K = sample_regression(best_candidate[0][0],
                                            best_candidate[0][1])
        #print("candidate: ", best_candidate)
        #print("model score for candidate: ", float(y_new_sample))

        X_samples = torch.vstack((X_samples, best_candidate))
        y_samples = torch.vstack((y_samples, y_new_sample))
    return y_samples


# params
n_samples = 20
n_bo = 15

plot = False

# measurements
improvement_mean = torch.zeros((n_bo+1, 1))
improvement_list = []
# bo sampling loop
p, ax = plt.subplots(figsize=(10, 10))

# only plot current best
# show simulated computation time on x-axis
for i in range(0, n_samples):
    print("::::::::::::::::::::::::::::::::::::::::::::::::::::")
    print("Sample ", i)
    result = sample_bo(n_bo, plot=plot)
    improvement = torch.tensor([result[0]])

    for j in range(1, len(result)):
        if improvement[-1] > result[j]:
            improvement = torch.vstack((improvement, result[j]))
        else:
            improvement = torch.vstack((improvement, improvement[-1]))

    improvement_mean += improvement
    improvement_list.append(improvement)
    ax.plot(np.arange(0, len(improvement), 1),
            improvement,
            '-.',
            color='blue',
            alpha = 0.3)

compute_time_steps = torch.arange(0, n_bo+1, 0.1)
improvement_list_scaled = []
for i in range(0, len(improvement_list)):
    current_improvement = improvement_list[i]
    current_compute_time = torch.arange(0, n_bo+1, 1)
    scaled_improvement = torch.zeros((len(compute_time_steps)), 1)
    iter = 0
    compute_time = 0
    for j in range(0, len(compute_time_steps)):
        imp = current_improvement[iter]
        if not iter + 1 > len(current_compute_time) -1:
            compute_time = current_compute_time[iter+1]
        scaled_improvement[j] = imp
        if compute_time_steps[j] >= compute_time:
            if iter < len(current_improvement)-1:
                iter += 1
    improvement_list_scaled.append(scaled_improvement)

# calculate mean
improvement_mean = torch.zeros((len(compute_time_steps), 1))
for improvement in improvement_list_scaled:
    improvement_mean += improvement
improvement_mean *= 1/n_samples

# calculate var
improvement_var = torch.zeros((len(improvement_mean), 1))
for i in range(0, n_samples):
    improvement_var += torch.square(improvement_list_scaled[i] - improvement_mean)
improvement_var *= 1/n_samples
improvement_var = torch.sqrt(improvement_var)

print("improvement_mean: ", improvement_mean)
print("improvement_var: ", improvement_var)

ax.plot(compute_time_steps,
        improvement_mean,
        color='red',
        label='mean')
ax.fill_between(compute_time_steps,
                (improvement_mean - improvement_var).flatten(),
                (improvement_mean + improvement_var).flatten(),
                alpha=0.3,
                color='red',
                label='standard deviation')
print("MEAN: ", improvement_mean)
print("VAR: ", improvement_var)
print("MEAN MAX: ", max(improvement_mean))
print("MEAN MIN: ", min(improvement_mean))
ax.set_xlabel("iteration")
ax.set_ylabel("y+")
ax.set_title("EI - best candidate in each iteration")
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.set_ylim([0., 0.6])
#ax.spines['right'].set_color('none')
#ax.spines['top'].set_color('none')
#ax.xaxis.set_ticks_position('bottom')
#ax.yaxis.set_ticks_position('left')

ax.legend()
plt.show()




::::::::::::::::::::::::::::::::::::::::::::::::::::
Sample  0
initialization
candidate:  tensor([0.4963, 0.7682])
model score for candidate:  0.46081276177468955
iteration:       0
iteration:       1
iteration:       2
iteration:       3
iteration:       4
iteration:       5
iteration:       6
iteration:       7
iteration:       8
iteration:       9
iteration:       10
iteration:       11
iteration:       12
iteration:       13
iteration:       14
::::::::::::::::::::::::::::::::::::::::::::::::::::
Sample  1
initialization
candidate:  tensor([0.5369, 0.5520])
model score for candidate:  0.40262656877965347
iteration:       0
iteration:       1
iteration:       2
iteration:       3
iteration:       4
iteration:       5
iteration:       6
iteration:       7
iteration:       8
iteration:       9
iteration:       10
iteration:       11
iteration:       12
iteration:       13
iteration:       14
::::::::::::::::::::::::::::::::::::::::::::::::::::
Sample  2
initialization
candidate:  tens